In [ ]:
# Imports
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import numpy as np
import pandas as pd
import time
import re
import os
import shutil

In [ ]:
# Folders os cpf/cnpjs that already exist
folders_cpf_cnpjs = [f for f in os.listdir() if re.match('^[0-9]+$',f)]

# List of cpf cnpj to execute
l_cpf_cnpj = []

# Create folder
[os.mkdir(c) for c in l_cpf_cnpj if not c in folders_cpf_cnpjs]

# Folders os cpf/cnpjs that already exist
folders_cpf_cnpjs = [f for f in os.listdir() if re.match('^[0-9]+$',f)]

In [ ]:
# Driver to start chrome
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
prefs = {"profile.default_content_settings.popups": 0,"download.default_directory": r"C:\Users\lukin\Desktop\Anoregmt\\", "directory_upgrade": True, "plugins.always_open_pdf_externally": True }
options.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(executable_path='C:/Users/lukin/Downloads/chromedriver', options=options)

# Get site
driver.get('https://app.anoregmt.org.br/#/login')

In [ ]:
# Define credentials
user = ''
password = ''

# Credentials
user_we = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div/div/div[1]/div/main/div[2]/div[2]/div[2]/label[1]/div/div/div/input')
user_we.send_keys(user)
password_we = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div/div/div[1]/div/main/div[2]/div[2]/div[2]/label[2]/div/div[1]/div[1]/input')
password_we.send_keys(password)

# Access page
access = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div/div/div[1]/div/main/div[2]/div[2]/div[3]/div/div[1]/button/span[2]/span')
access.click()

# Wait 5 seconds
time.sleep(5)

# Access "Pesquisa Dinamica"
pesquisa_dinamica = driver.find_element_by_xpath('//*[@id="q-app"]/div/div[1]/aside/div/div[1]/div[1]/div/div[2]/div[3]/div[3]/div')
pesquisa_dinamica.click()

# Wait 3 seconds
time.sleep(3)

# Input to cpf/cnpj
cpf_cnpj = driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div/div[2]/div/div/div/label/div/div[1]/div[2]/input')
cpf_cnpj.send_keys(l_cpf_cnpj[0])

# Click on "Pesquisar"
pesquisar = driver.find_element_by_xpath('//*[@id="q-app"]/div/div[2]/div/div[2]/div/div/div/label/div/div[1]/div[3]/button/span[2]/span/span')
pesquisar.click()

# Wait 3 seconds
time.sleep(3)

In [ ]:
# Result from search
result = driver.find_elements_by_xpath('//*[@id="q-app"]/div/div[2]/div/div[2]/div/div/div/div[1]/div[1]/table/tbody/tr/td[1]')

# Result's text
result_text = np.array([r.text for r in result])

# Index to get the right button position
index_result = np.where(result_text == 'Registro de Imóveis')[0]

if(len(index_result) != 0):
    # Click on "Registro de Imóveis"
    result[index_result[0]].click()
    
# Wait
time.sleep(3)

In [ ]:
def expand_pages(driver):
    
    # Get Number of pages
    number_of_pages = driver.find_element_by_xpath('//*[@id="q-app"]/div/div[2]/div/div[2]/div/div/div/div[1]/div[2]/span')
    number_of_pages = int(number_of_pages.text.split(' ')[2])

    # Expand all pages
    for i in range(number_of_pages-1):

        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Click on "Ver Mais" to open other registers
        ver_mais = driver.find_element_by_xpath('//*[@id="q-app"]/div/div[2]/div/div[2]/div/div/div/div[1]/div[3]/button/span[2]/span/span')
        ver_mais.click()

        # Wait 3 seconds
        time.sleep(3)

    # Result of search
    result = driver.find_elements_by_xpath('//*[@id="q-app"]/div/div[2]/div/div[2]/div/div/div/div[1]/div[1]/table/tbody/tr/td[3]')

    # Scroll to the page's start
    driver.execute_script("window.scrollTo(0, 0);")
    
    return result

def download_pdf(driver,register,error):
    try:
        
        # Scroll to next element
        driver.execute_script("arguments[0].scrollIntoView(true);", register)
        driver.execute_script("window.scrollBy(0,-50)")
        
        # Click to open a window
        register.click()

        # Wait 5 seconds
        time.sleep(5)

        # Click on button "visualizar"
        visualizar = driver.find_element_by_xpath('/html/body/div[5]/div[2]/div/div[4]/button[1]/span[2]/span/span')
        visualizar.click()

        # Wait 5 seconds
        time.sleep(5)

        if(len(driver.window_handles) > 1):
            # Switch to new tab created
            driver.switch_to.window(driver.window_handles[-1])

            # Close this new tab
            driver.close()

            # Back to original tab
            driver.switch_to.window(driver.window_handles[0])

        # Close window
        driver.find_element_by_css_selector('body').send_keys(Keys.CONTROL + Keys.ESCAPE)

        # Wait 3 seconds
        time.sleep(3)
    except:
        # Save and print error
        error.append(i)
        print(i)
        
        # Wait 3 seconds
        time.sleep(3)
        
        # Close window (if some was oppened)
        driver.find_element_by_css_selector('body').send_keys(Keys.CONTROL + Keys.ESCAPE)
        
def get_table_from_info(driver):
    
    # Get table
    ul = driver.find_element_by_xpath('/html/body/div[5]/div[2]/div/div[3]/div/div/div/div[1]/div/div/ul')

    # Convert to daframe
    table = pd.read_html(ul.get_attribute('innerHTML'))

    # Get title of each table
    title = np.array(ul.text.split('\n'))
    title = title[np.where(['PRATICADO' in t for t in title])[0] + 1]

    # Get date from each table
    date = driver.find_elements_by_xpath('/html/body/div[5]/div[2]/div/div[3]/div/div/div/div[1]/div/div/ul/li/div[1]/span')
    date = [d.text.split(' ')[2] for d in date]

    # Add new information to df
    for i in range(len(table)):
        table[i].columns = ['Partes','CPF_CNPJ','Nome']
        table[i]['Title'] = title[i]
        table[i]['Date'] = date[i]

    # Concat all dataframes
    table = pd.concat(table)
    
    return table


def get_info(driver,register,error):
    try:
        
        # Scroll to next element
        driver.execute_script("arguments[0].scrollIntoView(true);", register)
        driver.execute_script("window.scrollBy(0,-50)")
        
        # Click to open a window
        register.click()

        # Wait 3 seconds
        time.sleep(3)

        # Click on button "Informacoes"
        informacoes = driver.find_element_by_xpath('/html/body/div[5]/div[2]/div/div[2]/div/div[3]/div[2]')
        informacoes.click()

        # Wait 0.5 seconds
        time.sleep(0.5)
        
        # Get table
        table = get_table_from_info(driver)
        table.to_excel(re.sub('[^0-9]','',register.text.split('\n')[1]) + '.xlsx',index=False)
        
        # Close window
        driver.find_element_by_css_selector('body').send_keys(Keys.CONTROL + Keys.ESCAPE)
    except:
        # Save and print error
        error.append(i)
        print(i)
        
        # Wait 3 seconds
        time.sleep(3)
        
        # Close window (if some was oppened)
        driver.find_element_by_css_selector('body').send_keys(Keys.CONTROL + Keys.ESCAPE)

In [ ]:
# Expand registers
result = expand_pages(driver)

In [ ]:
# For each register
error = []
for i in range(len(result)):
    download_pdf(driver,result[i],error)

In [ ]:
# For each register (again)
run_again = error
error = []
for i in run_again:
    download_pdf(driver,result[i],error)

In [ ]:
# Move downloaded files to right folder
downloaded_files = [f for f in os.listdir() if re.match('^.*\.pdf',f)]
for d in downloaded_files:
    shutil.move(os.getcwd() + '\\' + d,os.getcwd() + '\\' + '' + '\\' + d)